In [ ]:
from chaosmagpy.plot_utils import nio_colormap
from pathlib import Path
import pooch
import holoviews as hv
import hvplot.xarray
import xarray as xr

## Prepare access to data

In [ ]:
data = {
    "path": Path("data"),
    "fname": "SwA_20140501-20190501_proc1.nc",
    "url": "https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1qX-j_QWx0OQTh1HLUlHHcWdNhFOsczyM",
    "known_hash": "1b7a8cbc0cb1657f8d4444ae7f6bbab91841318e1a172fa1f8a487b9d9492912",
}

In [ ]:
def download_data(data=data):
    full_path = data["path"] / data["fname"]
    if full_path.exists():
        return f"Already found file: {full_path}"
    else:
        return pooch.retrieve(
            url=data["url"],
            known_hash=data["known_hash"],
            path=data["path"],
            fname=data["fname"],
            progressbar=True,
        )

In [ ]:
download_data()

## Open data

In [ ]:
ds = xr.open_dataset(data["path"] / data["fname"])
ds

In [ ]:
# Dataset downsampled by 1/30 (i.e. 5-minute sampling)
_ds = ds.isel(Timestamp=slice(0, -1, 30))
# Generate residuals to plot
_ds["B_NEC_res_CHAOS-full"] = (
    _ds["B_NEC"]
    - _ds["B_NEC_CHAOS-MCO"]
    - _ds["B_NEC_CHAOS-MMA"]
    - _ds["B_NEC_CHAOS-Static_n16plus"]
)

## Visualise data with hvplot

In [ ]:
def plot_NEC_var(_ds=_ds, var="B_NEC", qdmlt=False, **kwargs):
    if qdmlt:
        x, y = "MLT", "QDLat"
    else:
        x, y = "Longitude", "Latitude"
    return (
        _ds.drop("Timestamp")
        .hvplot.scatter(
            x=x, y=y, c=var,
            by="NEC", subplots=True,
            rasterize=True,
            colorbar=True,
            hover=True,
            width=300, height=200,
            **kwargs
        )
    )


print("B_NEC: magnetic field measurements")
plot_NEC_var(_ds=_ds, var="B_NEC", clim=(-50000, 50000), cmap=nio_colormap())